In [7]:
import pandas as pd
d = {"headers":{"Users":["user_id","join_date","favorite_brand"],"Orders":["order_id","order_date","item_id","buyer_id","seller_id"],"Items":["item_id","item_brand"]},"rows":{"Users":[[1,"2019-01-01","Lenovo"],[2,"2019-02-09","Samsung"],[3,"2019-01-19","LG"],[4,"2019-05-21","HP"]],"Orders":[[1,"2019-08-01",4,1,2],[2,"2019-08-02",2,1,3],[3,"2019-08-03",3,2,3],[4,"2019-08-04",1,4,2],[5,"2019-08-04",1,3,4],[6,"2019-08-05",2,2,4]],"Items":[[1,"Samsung"],[2,"Lenovo"],[3,"LG"],[4,"HP"]]}}
pd.DataFrame(d['rows']['Orders'], columns=d['headers']['Orders']).to_csv("./Orders.txt", index=None)
pd.DataFrame(d['rows']['Users'], columns=d['headers']['Users']).to_csv("./Users.txt", index=None)
pd.DataFrame(d['rows']['Items'], columns=d['headers']['Items']).to_csv("./Items.txt", index=None)

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("martket analysis II").config("pyspark.sql.shuffle.partition", "4").getOrCreate()

In [9]:
o = spark.read.option("header", True).csv("./Orders.txt")
u = spark.read.option("header", True).csv("./Users.txt")
i = spark.read.option("header", True).csv("./Items.txt")
o.show()
u.show()
i.show()

+--------+----------+-------+--------+---------+
|order_id|order_date|item_id|buyer_id|seller_id|
+--------+----------+-------+--------+---------+
|       1|2019-08-01|      4|       1|        2|
|       2|2019-08-02|      2|       1|        3|
|       3|2019-08-03|      3|       2|        3|
|       4|2019-08-04|      1|       4|        2|
|       5|2019-08-04|      1|       3|        4|
|       6|2019-08-05|      2|       2|        4|
+--------+----------+-------+--------+---------+

+-------+----------+--------------+
|user_id| join_date|favorite_brand|
+-------+----------+--------------+
|      1|2019-01-01|        Lenovo|
|      2|2019-02-09|       Samsung|
|      3|2019-01-19|            LG|
|      4|2019-05-21|            HP|
+-------+----------+--------------+

+-------+----------+
|item_id|item_brand|
+-------+----------+
|      1|   Samsung|
|      2|    Lenovo|
|      3|        LG|
|      4|        HP|
+-------+----------+



In [6]:
o = o.withColumn("order_id", o.order_id.cast('int')).withColumn("seller_id", o.seller_id.cast('int')).withColumn("item_id", o.item_id.cast('int')).withColumn("buyer_id", o.buyer_id.cast('int')).withColumn("order_date", o.order_date.cast('date'))
u = u.withColumn("user_id", u.user_id.cast('int')).withColumn("join_date", u.join_date.cast('date'))

Write an SQL query to find for each user whether the brand of the second item (by date) they sold is their favorite brand. If a user sold less than two items, report the answer for that user as no. It is guaranteed that no seller sold more than one item on a day.

Return the result table in any order.

The query result format is in the following example.

In [16]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number,col,when
window_spec = Window.partitionBy("seller_id").orderBy("order_date")
second_item = o.join(i, "item_id", "inner").select("order_date", "seller_id", "item_brand", row_number().over(window_spec).alias("rn")).where(col('rn')==2)
u.join(second_item, second_item.seller_id==u.user_id, "leftouter").select("user_id", when(u.favorite_brand==second_item.item_brand, "yes").otherwise("no").alias("2nd_favori_brand")).na.fill("no").show()

+-------+----------------+
|user_id|2nd_favori_brand|
+-------+----------------+
|      1|              no|
|      2|             yes|
|      3|             yes|
|      4|              no|
+-------+----------------+

